# TOMAS ACOSTA BERNAL 
202011237

# Gridworld y su solución como MDPs

En este trabajo definiremos el ambiente de Gridworld y su solución como un MDP.
Gridworld es un ambiente clásico de prueba dentro del aprendizaje por refuerzo. Durante este taller definiremos el modelo básico del ambiente, que extenderemos incrementalmente de acuerdo a las necesidades del algoritmo de solución.

## Ambiente 🌎

El ambiente de gridworld se define como una cuadricula de `nxm`. El ambiente tiene obstaculos, es decir casillas por las cuales no puede pasar el agente. Al chocar con un obstaculo, el agente terminaría en el mismo estado inicial. Además, el ambiente tiene una casilla de inicio, y algunas casillas de salida. Un ejemplo del ambiente para el caso `3x4` se muestra a continuación.

![gridworld.png](./img/gridworld.png)

En este ejemplo del ambiente el agente comienza en la casilla inferior izquierda y tiene como objetivo llegar a la casilla de salida verde, con recompensa 1. La otra casilla de salida, tiene recompensa -1.



### Task 1.
#### ¿Cómo podemos codificar el ambiente?

De una definición completa del ambiente, como una clase de python llamada `Environment`, estableciendo:
1. Un atributo que define la cuadrícula (`board`). El ambiente recibirá una matriz como parámetro describiendo la cuadrícula en el momento de su creación. Definiremos las casillas por las que puede pasar el agente como casillas vacias, las casillas por las que no puede pasar el agente con un valor none `#` y las casillas de salida con el valor asociado a la recompensa definidas para cada una de ellas.
2. Un atributo `nrows` para almacenar la cantidad de filas de la cuadrícula.
3. Un atributo `ncols` para almacenar la cantidad de columnas de la cuadrícula.
4. Un atributo `initial_state` para almacenar el estado inicial del agente dentro del ambiente.
5. Un atributo con el estado actual (`current_state`) en el que se encuentra el agente. El valor de `current_state` se definirá como una tupla 
6. Un atributo `P` que guarda la matriz de probabilidades de cada una de las acciones para cada estado. Dicha matriz esta definida por parámetro.

Un ejemplo de la definición del tablero para el caso de 5x5 de la figura anterior se da a continuación.
```
board = [['', ' ', ' ',  '+1'],
         [' ', '#', ' ',  '-1'],
         ['S', ' ', ' ', ' ']]
```
En el ejemplo `S` denota el estado inicial y `'#'` la casilla prohibida (manejaremos esta convención para todos los ambientes de gridworld).

De forma similar a la definición del `board` la matriz de probabilidades `P` se define como:
```
P = [[[0.1, 0.1, 0, 0.8], [0.1, 0.1, 0, 0.8], [0.1, 0.1, 0, 0.8],  [1]],
         [[0.8, 0, 0.1, 0.1], '#', [0.8, 0, 0.1, 0.1],  [-1]],
         [[0.8, 0, 0.1, 0.1], [0.1, 0.1, 0.8, 0], [0.1, 0.1, 0.8, 0], [0.1, 0.1, 0.8, 0]]]
```
Para la definición de `P` vamos a entender cada una de las posiciones de la probabilidad en el orden (`'up', 'down', 'left', 'right'`). Adicionalmente, vamos a suponer que la casilla da la probabilidad de tal forma que el agente siempre toma la acción en dirección al objetivo (la acción que tiene probabilidad `0.8`). Por ejemplo, para la casilla superior izquierda la probabilidad de tomar la acción `up` y llegar a la casilla de arriba es de `0.1`, a la casilla de abajo con probabilidad `0.1` y a la casilla de la derecha con  probabilidad `0.8` (el agente nunca puede llegar a la casilla de la izquierda). En las casillas de salida, el agente solo tiene una posibilidad que es tomar la acción `exit` que le da la recompensa asociada a la casilla al agente.


#### Comportamiento del ambeinte

Una vez definido el ambiente definimos su comportamiento. Para ello requerimos los siguientes métodos:
1. `get_current_state` que no recibe parámetros y retorna el estado actual (la casilla donde se encuentra el agente)
2. `get_posible_actions` que recibe el estado actual del agente como parámetro y retorna las acciones disponibles para dicho estado. Las acciones estarán dadas por su nombre (`'up', 'down', 'left', 'right'`) para las casillas normales y (`'exit'`) para las casillas de salida. Como convención definiremos que el agente siempre puede moverse en todas las direcciones, donde un movimiento en dirección de un obstáculo o los límites del ambiente no tienen ningún efecto visible en la posición del agente.
3. `do_action` que recibe como parámetro la acción a ejecutar y retorna el valor de la recompensa y el nuevo estado del agente, como un pareja `reward, new_state`. Note que `do_action` esta restringida por la matriz de probabilidad `P` para la ejecución real de las acciones.
4. `reset` que no recibe parámetros y restablece el ambiente a su estado inicial.
5. `is_terminal` que no recibe parámetros y determina si el agente está en el estado final o no. En nuestro caso, el estado final estará determinado por las casillas de salida (i.e., con un valor definido).



In [1]:
from loguru import logger
import numpy as np
from typing import Tuple, List, Union


class Environment:
    def __init__(self, board: List[List[Union[str, int, float]]], P: List[List[Union[str, List[float]]]], initial_state: Tuple[int, int]):
        logger.info("Inicializando ambiente de Gridworld")
        
        self.board = board
        self.nrows = len(board)
        self.ncols = len(board[0])
        self.initial_state = initial_state
        self.current_state = initial_state
        self.P = P
        
        logger.debug(f"Dimensiones del tablero: {self.nrows}x{self.ncols}")
        logger.debug(f"Estado inicial: {self.initial_state}")
        logger.success("Ambiente inicializado correctamente")
    
    def get_current_state(self) -> Tuple[int, int]:
        logger.trace(f"Obteniendo estado actual: {self.current_state}")
        return self.current_state
    
    def get_possible_actions(self, state: Tuple[int, int]) -> List[str]:
        r, c = state
        
        logger.trace(f"Obteniendo acciones posibles para estado {state}")
        
        if isinstance(self.board[r][c], (int, float)):
            logger.debug(f"Estado {state} es terminal, acción disponible: ['exit']")
            return ['exit']
        
        actions = ['up', 'down', 'left', 'right']
        logger.debug(f"Estado {state} tiene acciones: {actions}")
        return actions
    
    def do_action(self, action: str) -> Tuple[Union[int, float], Tuple[int, int]]:
        r, c = self.current_state
        
        logger.info(f"Ejecutando acción '{action}' desde estado {self.current_state}")
        
        if self.is_terminal():
            if action == 'exit':
                reward = self.board[r][c]
                logger.success(f"Acción 'exit' ejecutada. Recompensa: {reward}")
                return reward, self.current_state
            else:
                logger.warning(f"Acción '{action}' inválida en estado terminal. Sin efecto.")
                return 0, self.current_state
        
        if self.P[r][c] == '#':
            logger.warning(f"Estado {self.current_state} es un obstáculo. Sin efecto.")
            return 0, self.current_state
        
        probs = self.P[r][c]
        
        action_map = {'up': 0, 'down': 1, 'left': 2, 'right': 3}
        actual_action_idx = np.random.choice(4, p=probs)
        actual_actions = ['up', 'down', 'left', 'right']
        actual_action = actual_actions[actual_action_idx]
        
        if actual_action != action:
            logger.debug(f"Estocasticidad: acción solicitada '{action}', acción ejecutada '{actual_action}'")
        
        new_state = self._calculate_new_state(r, c, actual_action)
        
        reward = 0
        if isinstance(self.board[new_state[0]][new_state[1]], (int, float)):
            reward = self.board[new_state[0]][new_state[1]]
            logger.info(f"Estado terminal alcanzado {new_state} con recompensa {reward}")
        
        self.current_state = new_state
        logger.info(f"Nuevo estado: {self.current_state}, Recompensa: {reward}")
        
        return reward, new_state
    
    def _calculate_new_state(self, r: int, c: int, action: str) -> Tuple[int, int]:
        logger.trace(f"Calculando nuevo estado desde ({r}, {c}) con acción '{action}'")
        
        if action == 'up':
            new_r, new_c = r - 1, c
        elif action == 'down':
            new_r, new_c = r + 1, c
        elif action == 'left':
            new_r, new_c = r, c - 1
        elif action == 'right':
            new_r, new_c = r, c + 1
        else:
            logger.error(f"Acción desconocida: '{action}'")
            return (r, c)
        
        if (new_r < 0 or new_r >= self.nrows or 
            new_c < 0 or new_c >= self.ncols or
            self.board[new_r][new_c] == '#'):
            logger.debug(f"Movimiento bloqueado (límite/obstáculo). Permanece en ({r}, {c})")
            return (r, c)
        
        logger.trace(f"Nuevo estado calculado: ({new_r}, {new_c})")
        return (new_r, new_c)
    
    def reset(self) -> None:
        logger.info("Reseteando ambiente al estado inicial")
        self.current_state = self.initial_state
        logger.success(f"Ambiente reseteado. Estado actual: {self.current_state}")
    
    def is_terminal(self) -> bool:
        r, c = self.current_state
        is_term = isinstance(self.board[r][c], (int, float))
        logger.trace(f"Verificando si estado {self.current_state} es terminal: {is_term}")
        return is_term


if __name__ == "__main__":
    logger.add("gridworld.log", rotation="500 MB", level="DEBUG")
    
    board = [['', ' ', ' ', 1],
             [' ', '#', ' ', -1],
             ['S', ' ', ' ', ' ']]
    
    P = [[[0.1, 0.1, 0, 0.8], [0.1, 0.1, 0, 0.8], [0.1, 0.1, 0, 0.8], [1, 0, 0, 0]],
         [[0.8, 0, 0.1, 0.1], '#', [0.8, 0, 0.1, 0.1], [0, 1, 0, 0]],
         [[0.8, 0, 0.1, 0.1], [0.1, 0.1, 0.8, 0], [0.1, 0.1, 0.8, 0], [0.1, 0.1, 0.8, 0]]]
    
    initial_state = (2, 0)
    
    env = Environment(board, P, initial_state)
    
    logger.info("=== Prueba de ambiente ===")
    logger.info(f"Estado actual: {env.get_current_state()}")
    logger.info(f"Acciones posibles: {env.get_possible_actions(env.get_current_state())}")
    
    for i in range(5):
        logger.info(f"\n--- Step {i+1} ---")
        action = np.random.choice(['up', 'down', 'left', 'right'])
        reward, new_state = env.do_action(action)
        logger.info(f"Acción tomada: {action}, Recompensa: {reward}, Nuevo estado: {new_state}")
        
        if env.is_terminal():
            logger.success("Estado terminal alcanzado!")
            break
    
    env.reset()
    logger.info(f"Después del reset, estado actual: {env.get_current_state()}")

2026-02-16 20:13:17.016 | INFO     | __main__:__init__:8 - Inicializando ambiente de Gridworld
2026-02-16 20:13:17.017 | DEBUG    | __main__:__init__:17 - Dimensiones del tablero: 3x4
2026-02-16 20:13:17.017 | DEBUG    | __main__:__init__:18 - Estado inicial: (2, 0)
2026-02-16 20:13:17.018 | SUCCESS  | __main__:__init__:19 - Ambiente inicializado correctamente
2026-02-16 20:13:17.018 | INFO     | __main__:<module>:129 - === Prueba de ambiente ===
2026-02-16 20:13:17.019 | INFO     | __main__:<module>:130 - Estado actual: (2, 0)
2026-02-16 20:13:17.020 | DEBUG    | __main__:get_possible_actions:35 - Estado (2, 0) tiene acciones: ['up', 'down', 'left', 'right']
2026-02-16 20:13:17.022 | INFO     | __main__:<module>:131 - Acciones posibles: ['up', 'down', 'left', 'right']
2026-02-16 20:13:17.052 | INFO     | __main__:<module>:134 - 
--- Step 1 ---
2026-02-16 20:13:17.054 | INFO     | __main__:do_action:41 - Ejecutando acción 'right' desde estado (2, 0)
2026-02-16 20:13:17.056 | DEBUG    |

Teniendo en cuenta la definición del agente, genere un ambiente de `10x10` como se muestra a continuación.

![evaluacion.png](./img/evaluacion.png)

### Task 2.
Plantee el problema de MDP para cada una de las casillas. Especifique el estado de inicio, las transiciones y su probabilidad (suponiendo que todas las acciones sucede con probabilidad de 0.25) y los estados de fin con su recompensa.
¿Cómo serían las recompensas esperadas para cada estado?



# Task 2: Planteamiento del MDP para el ambiente 10x10

## Definición del ambiente


- **Estado inicial**: Casilla superior izquierda (0, 0) 
- **Obstáculos**: Casillas grises que forman una barrera vertical y horizontal
- **Estados terminales**:
  - (4, 5): Recompensa R = -1
  - (5, 5): Recompensa R = 1
  - (7, 4): Recompensa R = -1
  - (7, 5): Recompensa R = -1

## Componentes del MDP

### 1. S

El conjunto de estados está compuesto por todas las casillas del grid 10×10, excluyendo los obstáculos:
```
S = {(i, j) | 0 ≤ i < 10, 0 ≤ j < 10, (i, j) ∉ Obstáculos}
```

Donde los **Obstáculos** son:
```
Obstáculos = {
    (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7),
    (3, 4),
    (4, 4),
    (5, 4),
    (6, 4),
    (7, 4) - estado terminal,
    (8, 4)
}
```

Y los **Estados Terminales** son:
```
S_terminal = {(4, 5), (5, 5), (7, 4), (7, 5)}
```

### 2. A

Para cada estado no terminal:
```
A(s) = {up, down, left, right}  ∀s ∈ S \ S_terminal
```

Para estados terminales:
```
A(s) = {exit}  ∀s ∈ S_terminal
```

### 3. P

Dado que todas las acciones ocurren con **probabilidad uniforme de 0.25**, la función de transición se define como:

Para cualquier estado no terminal `s = (r, c)`:
```
P(s' | s, a) = 0.25  para cada a ∈ {up, down, left, right}
```

Donde `s'` es el estado resultante después de ejecutar la acción `a`.

#### Reglas para poder hacer la transición:

1. Movimiento válido: Si la acción conduce a una casilla válida (dentro del grid y sin obstáculo), el agente se mueve a esa casilla.

2. Colisión con obstáculo o límite: Si la acción conduce a un obstáculo o fuera del grid, el agente permanece en el estado actual.

3. Estados terminales: Una vez alcanzado un estado terminal, el agente permanece allí.


### 4. R

La función de recompensa se define como:
```
R(s, a, s') = {
    1     si s' = (5, 5)
   -1     si s' ∈ {(4, 5), (7, 4), (7, 5)}
    0     en caso contrario
}
```

O de manera equivalente:
```
R(s) = {
    1     si s = (5, 5)
   -1     si s ∈ {(4, 5), (7, 4), (7, 5)}
    0     para todos los demás estados
}
```



## Recompensas Esperadas para Cada Estado

La recompensa esperada para cada estado depende de la política seguida. Sin embargo, podemos calcular las recompensas inmediatas esperadas asumiendo acciones equiprobables:

### Estados no terminales

Para cualquier estado `s` no terminal, la recompensa esperada inmediata bajo una política uniforme aleatoria es:
```
E[R | s] = Σ P(a) Σ P(s' | s, a) R(s, a, s')
         = 0.25 × Σ_{a ∈ A} Σ_{s'} P(s' | s, a) R(s, a, s')
```

**Ejemplo: Estado (4, 4)**
```
Acciones posibles: {up, down, left, right}

up:    s' = (4, 4)     R = 0
down:  s' = (4, 4)     R = 0
left:  s' = (4, 3)     R = 0
right: s' = (4, 5)     R = -1

E[R | (4,4)] = 0.25 × (0 + 0 + 0 + (-1)) = -0.25
```

**Ejemplo: Estado (4, 6)**
```
Acciones posibles: {up, down, left, right}

up:    s' = (3, 6)     R = 0
down:  s' = (5, 6)     R = 0
left:  s' = (4, 5)     R = -1
right: s' = (4, 7)     R = 0

E[R | (4,6)] = 0.25 × (0 + 0 + (-1) + 0) = -0.25
```

**Ejemplo: Estado (5, 6)**
```
Acciones posibles: {up, down, left, right}

up:    s' = (4, 6)     R = 0
down:  s' = (6, 6)     R = 0
left:  s' = (5, 5)     R = 1
right: s' = (5, 7)     R = 0

E[R | (5,6)] = 0.25 × (0 + 0 + 1 + 0) = 0.25
```

### Estados terminales

Para estados terminales, la recompensa esperada es simplemente la recompensa del estado:
```
E[R | (4, 5)] = -1
E[R | (5, 5)] = 1
E[R | (7, 4)] = -1
E[R | (7, 5)] = -1
```

### Task 3.
Bajo la definción del problema anterior, suponga que cada acción tiene una probabilidad de éxito de 60%, con probabilidad de 20% se ejecutará la sigiente acción (en dirección de las manesillas del reloj), con probabilidad de 10% se ejecutará la sigiente acción (en contra de las manesillas del reloj) y con probabilidad de 10% no pasará nada. Bajo estas condiciones, ¿Cómo serían las recompensas esperadas para cada estado? 

Codifique el ambiente para el gridworld de `10x10` utilizando esta función de probabilidad. En esta codificación del ambiente no es necesario pasar la matriz `P` como parámetro, pero esta información se debe tener en cuenta en la función `do_action`.

Tenga en cuenta que la calidad del programa que entreguen será tenida en cuentra dentro de la calificación.


In [4]:
from loguru import logger
import numpy as np
from typing import Tuple, List, Dict


class GridWorld10x10:
    def __init__(self):
        logger.info("Inicializando ambiente GridWorld 10x10")
        
        self.nrows = 10
        self.ncols = 10
        self.initial_state = (0, 0)
        self.current_state = self.initial_state
        
        self._initialize_board()
        self._initialize_obstacles()
        
        self.action_success_prob = 0.60
        self.clockwise_prob = 0.20
        self.counterclockwise_prob = 0.10
        self.stay_prob = 0.10
        
        self.action_map = {
            'up': (-1, 0),
            'down': (1, 0),
            'left': (0, -1),
            'right': (0, 1)
        }
        
        logger.success("Ambiente inicializado correctamente")
        logger.debug(f"Dimensiones: {self.nrows}x{self.ncols}")
        logger.debug(f"Estado inicial: {self.initial_state}")
        logger.debug(f"Probabilidades - Éxito: {self.action_success_prob}, Horario: {self.clockwise_prob}, Antihorario: {self.counterclockwise_prob}, Permanencia: {self.stay_prob}")
    
    def _initialize_board(self):
        self.board = [[' ' for _ in range(self.ncols)] for _ in range(self.nrows)]
        
        self.board[0][0] = 'S'
        
        self.board[4][5] = -1
        self.board[5][5] = 1
        self.board[7][4] = -1
        self.board[7][5] = -1
        
        logger.debug("Tablero inicializado con estados terminales")
    
    def _initialize_obstacles(self):
        obstacles = [
            (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7),
            (3, 4),
            (4, 4),
            (5, 4),
            (6, 4),
            (8, 4)
        ]
        
        for r, c in obstacles:
            self.board[r][c] = '#'
        
        logger.debug(f"Obstáculos inicializados: {len(obstacles)} casillas bloqueadas")
    
    def _get_clockwise_action(self, action: str) -> str:
        clockwise_map = {
            'up': 'right',
            'right': 'down',
            'down': 'left',
            'left': 'up'
        }
        return clockwise_map.get(action, action)
    
    def _get_counterclockwise_action(self, action: str) -> str:
        counterclockwise_map = {
            'up': 'left',
            'left': 'down',
            'down': 'right',
            'right': 'up'
        }
        return counterclockwise_map.get(action, action)
    
    def _calculate_new_state(self, r: int, c: int, action: str) -> Tuple[int, int]:
        if action not in self.action_map:
            logger.warning(f"Acción desconocida: '{action}'")
            return (r, c)
        
        dr, dc = self.action_map[action]
        new_r, new_c = r + dr, c + dc
        
        if (new_r < 0 or new_r >= self.nrows or 
            new_c < 0 or new_c >= self.ncols or
            self.board[new_r][new_c] == '#'):
            logger.trace(f"Movimiento bloqueado desde ({r},{c}) hacia '{action}'. Permanece en lugar")
            return (r, c)
        
        logger.trace(f"Movimiento válido desde ({r},{c}) hacia ({new_r},{new_c}) con acción '{action}'")
        return (new_r, new_c)
    
    def get_current_state(self) -> Tuple[int, int]:
        logger.trace(f"Estado actual: {self.current_state}")
        return self.current_state
    
    def get_possible_actions(self, state: Tuple[int, int]) -> List[str]:
        r, c = state
        
        if isinstance(self.board[r][c], (int, float)):
            logger.debug(f"Estado {state} es terminal, acción: ['exit']")
            return ['exit']
        
        actions = ['up', 'down', 'left', 'right']
        logger.debug(f"Estado {state} acciones disponibles: {actions}")
        return actions
    
    def do_action(self, action: str) -> Tuple[float, Tuple[int, int]]:
        r, c = self.current_state
        
        logger.info(f"Ejecutando acción '{action}' desde estado {self.current_state}")
        
        if self.is_terminal():
            if action == 'exit':
                reward = self.board[r][c]
                logger.success(f"Acción 'exit' ejecutada. Recompensa: {reward}")
                return reward, self.current_state
            else:
                logger.warning(f"Acción '{action}' inválida en estado terminal")
                return 0, self.current_state
        
        if action not in ['up', 'down', 'left', 'right']:
            logger.error(f"Acción desconocida: '{action}'")
            return 0, self.current_state
        
        clockwise_action = self._get_clockwise_action(action)
        counterclockwise_action = self._get_counterclockwise_action(action)
        
        actions_with_probs = [
            (action, self.action_success_prob),
            (clockwise_action, self.clockwise_prob),
            (counterclockwise_action, self.counterclockwise_prob),
            ('stay', self.stay_prob)
        ]
        
        rand = np.random.random()
        cumulative_prob = 0
        executed_action = action
        
        for act, prob in actions_with_probs:
            cumulative_prob += prob
            if rand < cumulative_prob:
                executed_action = act
                break
        
        if executed_action == 'stay':
            new_state = self.current_state
            logger.debug(f"Acción 'stay' ejecutada. Permanece en {self.current_state}")
        else:
            new_state = self._calculate_new_state(r, c, executed_action)
        
        if executed_action != action:
            logger.debug(f"Estocasticidad: intención '{action}' → ejecución '{executed_action}'")
        
        reward = 0
        if isinstance(self.board[new_state[0]][new_state[1]], (int, float)):
            reward = self.board[new_state[0]][new_state[1]]
            logger.info(f"Estado terminal alcanzado {new_state} con recompensa {reward}")
        
        self.current_state = new_state
        logger.info(f"Nuevo estado: {self.current_state}, Recompensa: {reward}")
        
        return reward, new_state
    
    def reset(self) -> None:
        logger.info("Reseteando ambiente al estado inicial")
        self.current_state = self.initial_state
        logger.success(f"Ambiente reseteado. Estado: {self.current_state}")
    
    def is_terminal(self) -> bool:
        r, c = self.current_state
        is_term = isinstance(self.board[r][c], (int, float))
        logger.trace(f"¿Estado {self.current_state} es terminal? {is_term}")
        return is_term
    
    def get_expected_reward(self, state: Tuple[int, int], action: str) -> float:
        r, c = state
        
        if isinstance(self.board[r][c], (int, float)):
            return self.board[r][c]
        
        if action not in ['up', 'down', 'left', 'right']:
            return 0.0
        
        clockwise_action = self._get_clockwise_action(action)
        counterclockwise_action = self._get_counterclockwise_action(action)
        
        actions_with_probs = [
            (action, self.action_success_prob),
            (clockwise_action, self.clockwise_prob),
            (counterclockwise_action, self.counterclockwise_prob),
            ('stay', self.stay_prob)
        ]
        
        expected_reward = 0.0
        
        for act, prob in actions_with_probs:
            if act == 'stay':
                next_state = (r, c)
            else:
                next_state = self._calculate_new_state(r, c, act)
            
            next_r, next_c = next_state
            if isinstance(self.board[next_r][next_c], (int, float)):
                reward = self.board[next_r][next_c]
            else:
                reward = 0
            
            expected_reward += prob * reward
        
        logger.trace(f"E[R | {state}, {action}] = {expected_reward:.3f}")
        return expected_reward
    
    def print_board(self) -> None:
        logger.info("Imprimiendo estado del tablero")
        print("\n" + "="*50)
        print("GRIDWORLD 10x10")
        print("="*50)
        
        for i in range(self.nrows):
            row_str = ""
            for j in range(self.ncols):
                if (i, j) == self.current_state:
                    row_str += "[A]"
                elif self.board[i][j] == '#':
                    row_str += " # "
                elif self.board[i][j] == 'S':
                    row_str += " S "
                elif isinstance(self.board[i][j], (int, float)):
                    row_str += f"{self.board[i][j]:+2} "
                else:
                    row_str += " . "
            print(row_str)
        
        print("="*50)
        print(f"Estado actual: {self.current_state}")
        print(f"Terminal: {self.is_terminal()}")
        print("="*50 + "\n")


if __name__ == "__main__":
    logger.add("gridworld_10x10.log", level="DEBUG")
    
    env = GridWorld10x10()
    
    logger.info("=== Visualización del tablero ===")
    env.print_board()
    
    logger.info("=== Análisis de recompensas esperadas ===")
    
    test_states = [
        ((0, 0), "Estado inicial"),
        ((4, 4), "Adyacente a terminal negativo"),
        ((5, 6), "Adyacente a terminal positivo"),
        ((2, 0), "Adyacente a obstáculos")
    ]
    
    for state, description in test_states:
        logger.info(f"\n--- {description}: {state} ---")
        for action in ['up', 'down', 'left', 'right']:
            expected_r = env.get_expected_reward(state, action)
            print(f"E[R | {state}, {action:>5}] = {expected_r:+.3f}")
    
    logger.info("\n=== Simulación de episodio ===")
    env.reset()
    
    total_reward = 0
    steps = 0
    max_steps = 1000
    
    while not env.is_terminal() and steps < max_steps:
        state = env.get_current_state()
        action = np.random.choice(['up', 'down', 'left', 'right'])
        
        reward, new_state = env.do_action(action)
        total_reward += reward
        steps += 1
        
        logger.info(f"Step {steps}: {state} --[{action}]--> {new_state}, R={reward:+.1f}")
    
    logger.success(f"\nEpisodio terminado en {steps} pasos")
    logger.success(f"Recompensa total acumulada: {total_reward:+.2f}")
    logger.success(f"Estado final: {env.get_current_state()}")
    
    env.print_board()

2026-02-16 20:20:45.198 | INFO     | __main__:__init__:8 - Inicializando ambiente GridWorld 10x10
2026-02-16 20:20:45.199 | DEBUG    | __main__:_initialize_board:45 - Tablero inicializado con estados terminales
2026-02-16 20:20:45.200 | DEBUG    | __main__:_initialize_obstacles:60 - Obstáculos inicializados: 12 casillas bloqueadas
2026-02-16 20:20:45.203 | SUCCESS  | __main__:__init__:30 - Ambiente inicializado correctamente
2026-02-16 20:20:45.203 | DEBUG    | __main__:__init__:31 - Dimensiones: 10x10


2026-02-16 20:20:45.204 | DEBUG    | __main__:__init__:32 - Estado inicial: (0, 0)
2026-02-16 20:20:45.205 | DEBUG    | __main__:__init__:33 - Probabilidades - Éxito: 0.6, Horario: 0.2, Antihorario: 0.1, Permanencia: 0.1
2026-02-16 20:20:45.206 | INFO     | __main__:<module>:250 - === Visualización del tablero ===
2026-02-16 20:20:45.208 | INFO     | __main__:print_board:219 - Imprimiendo estado del tablero
2026-02-16 20:20:45.209 | INFO     | __main__:<module>:253 - === Análisis de recompensas esperadas ===
2026-02-16 20:20:45.211 | INFO     | __main__:<module>:263 - 
--- Estado inicial: (0, 0) ---
2026-02-16 20:20:45.213 | INFO     | __main__:<module>:263 - 
--- Adyacente a terminal negativo: (4, 4) ---
2026-02-16 20:20:45.214 | INFO     | __main__:<module>:263 - 
--- Adyacente a terminal positivo: (5, 6) ---
2026-02-16 20:20:45.216 | INFO     | __main__:<module>:263 - 
--- Adyacente a obstáculos: (2, 0) ---
2026-02-16 20:20:45.217 | INFO     | __main__:<module>:268 - 
=== Simulación


GRIDWORLD 10x10
[A] .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  #  #  #  #  #  #  #  .  . 
 .  .  .  .  #  .  .  .  .  . 
 .  .  .  .  # -1  .  .  .  . 
 .  .  .  .  # +1  .  .  .  . 
 .  .  .  .  #  .  .  .  .  . 
 .  .  .  . -1 -1  .  .  .  . 
 .  .  .  .  #  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
Estado actual: (0, 0)
Terminal: False

E[R | (0, 0),    up] = +0.000
E[R | (0, 0),  down] = +0.000
E[R | (0, 0),  left] = +0.000
E[R | (0, 0), right] = +0.000
E[R | (4, 4),    up] = -0.200
E[R | (4, 4),  down] = -0.100
E[R | (4, 4),  left] = +0.000
E[R | (4, 4), right] = -0.600
E[R | (5, 6),    up] = +0.100
E[R | (5, 6),  down] = +0.200
E[R | (5, 6),  left] = +0.600
E[R | (5, 6), right] = +0.000
E[R | (2, 0),    up] = +0.000
E[R | (2, 0),  down] = +0.000
E[R | (2, 0),  left] = +0.000
E[R | (2, 0), right] = +0.000


2026-02-16 20:20:45.403 | DEBUG    | __main__:do_action:157 - Estocasticidad: intención 'up' → ejecución 'left'
2026-02-16 20:20:45.405 | INFO     | __main__:do_action:165 - Nuevo estado: (0, 3), Recompensa: 0
2026-02-16 20:20:45.406 | INFO     | __main__:<module>:283 - Step 7: (0, 4) --[up]--> (0, 3), R=+0.0
2026-02-16 20:20:45.409 | INFO     | __main__:do_action:115 - Ejecutando acción 'left' desde estado (0, 3)
2026-02-16 20:20:45.410 | INFO     | __main__:do_action:165 - Nuevo estado: (0, 2), Recompensa: 0
2026-02-16 20:20:45.411 | INFO     | __main__:<module>:283 - Step 8: (0, 3) --[left]--> (0, 2), R=+0.0
2026-02-16 20:20:45.412 | INFO     | __main__:do_action:115 - Ejecutando acción 'up' desde estado (0, 2)
2026-02-16 20:20:45.413 | INFO     | __main__:do_action:165 - Nuevo estado: (0, 2), Recompensa: 0
2026-02-16 20:20:45.414 | INFO     | __main__:<module>:283 - Step 9: (0, 2) --[up]--> (0, 2), R=+0.0
2026-02-16 20:20:45.415 | INFO     | __main__:do_action:115 - Ejecutando acci


GRIDWORLD 10x10
 S  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  #  #  #  #  #  #  #  .  . 
 .  .  .  .  #  .  .  .  .  . 
 .  .  .  .  # -1  .  .  .  . 
 .  .  .  .  # +1  .  .  .  . 
 .  .  .  .  #  .  .  .  .  . 
 .  .  .  . [A]-1  .  .  .  . 
 .  .  .  .  #  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
Estado actual: (7, 4)
Terminal: True




### Task 4. 
Defina una situación de la vide real (de su escogencia) como un MDP.

# MDP: Optimización de Prompts para Agentes de Servicio al Cliente

## Contexto del Problema

Cuando trabajo en agentes conversacionales, constantemente necesito iterar y optimizar prompts para mejorar la calidad de las respuestas del agente. Este proceso de optimización puede modelarse como un MDP donde el objetivo es alcanzar un prompt de alta calidad minimizando el tiempo y costo de iteraciones.

## Componentes del MDP

### 1. Estados (S)

El estado del sistema se representa como una tupla: `s = (quality_score, iteration_count, prompt_structure)`

Donde:
- **quality_score**: Calidad actual del prompt medida en el rango [0, 100]
  - 0-30: Pobre (respuestas inconsistentes, errores frecuentes)
  - 31-60: Aceptable (funciona pero necesita mejoras)
  - 61-85: Bueno (cumple requisitos básicos)
  - 86-100: Excelente (producción-ready)

- **iteration_count**: Número de iteraciones realizadas [0, 1, 2, 3, 4, 5+]

- **prompt_structure**: Estado actual de la estructura del prompt
  - `basic`: Instrucciones simples sin ejemplos
  - `with_examples`: Incluye 2-3 ejemplos
  - `structured`: Con formato XML/JSON y secciones claras
  - `advanced`: Incluye chain-of-thought, pocos ejemplos, manejo de errores

**Ejemplo de estados:**
```
s1 = (25, 0, 'basic')           # Prompt inicial pobre
s2 = (45, 1, 'with_examples')   # Primera iteración con ejemplos
s3 = (72, 2, 'structured')      # Segunda iteración estructurada
s4 = (88, 3, 'advanced')        # Prompt optimizado
```

**Estados terminales:**
- `quality_score >= 85`: Prompt aprobado para producción
- `iteration_count >= 6`: Límite de iteraciones alcanzado (requiere replantear estrategia)

### 2. Acciones (A)

Las acciones disponibles representan diferentes estrategias de optimización:
```
A = {
    'add_examples',          # Agregar 2-3 ejemplos de few shot
    'restructure',           # Cambiar formato (a XML, JSON, con secciones)
    'adjust_temperature',    # Modificar temperatura
    'add_constraints',       # Agregar reglas/restricciones específicas
    'add_cot',              # Agregar chain-of-thought reasoning
    'simplify',             # Simplificar prompt 
    'test_edge_cases',      # Evaluar
    'deploy'                # Desplegar a producción (solo si quality >= 85)
}
```

**Restricciones por estado:**
- Si `prompt_structure = 'basic'`: No puedes hacer `simplify`
- Si `quality_score < 85`: No puedes hacer `deploy`
- Si `iteration_count >= 6`: Solo puedes hacer `deploy` o reiniciar

### 3. Función de Transición P(s' | s, a)

La transición es estocástica porque el resultado de modificar un prompt tiene incertidumbre inherente.

#### Ejemplo 1: `add_examples` desde estado `(45, 1, 'basic')`
```
Acción: add_examples
Estado actual: (45, 1, 'basic')

Posibles resultados:
- (65, 2, 'with_examples'): prob = 0.60  # Mejora significativa
- (55, 2, 'with_examples'): prob = 0.25  # Mejora moderada
- (48, 2, 'with_examples'): prob = 0.10  # Mejora mínima
- (40, 2, 'with_examples'): prob = 0.05  # Empeora (ejemplos confusos)
```

#### Ejemplo 2: `restructure` desde estado `(55, 2, 'with_examples')`
```
Acción: restructure
Estado actual: (55, 2, 'with_examples')

Posibles resultados:
- (75, 3, 'structured'): prob = 0.50  # Estructura ayuda mucho
- (62, 3, 'structured'): prob = 0.30  # Mejora moderada
- (50, 3, 'structured'): prob = 0.15  # Poca mejora
- (45, 3, 'structured'): prob = 0.05  # Estructura muy compleja
```

#### Ejemplo 3: `adjust_temperature` desde estado `(72, 2, 'structured')`
```
Acción: adjust_temperature
Estado actual: (72, 2, 'structured')

Posibles resultados:
- (82, 3, 'structured'): prob = 0.40  # Temperatura óptima encontrada
- (75, 3, 'structured'): prob = 0.35  # Mejora leve
- (70, 3, 'structured'): prob = 0.20  # Sin cambio significativo
- (65, 3, 'structured'): prob = 0.05  # Temperatura incorrecta
```

#### Ejemplo 4: `add_cot` desde estado `(75, 3, 'structured')`
```
Acción: add_cot
Estado actual: (75, 3, 'structured')

Posibles resultados:
- (90, 4, 'advanced'): prob = 0.55  # CoT mejora razonamiento
- (82, 4, 'advanced'): prob = 0.30  # Mejora moderada
- (72, 4, 'advanced'): prob = 0.10  # Poco efecto
- (68, 4, 'advanced'): prob = 0.05  # CoT añade confusión
```

### 4. Función de Recompensa R(s, a, s')

La recompensa refleja el **trade-off entre calidad y costo/tiempo**:
```python
def R(s, a, s_prime):
    quality_old, iter_old, struct_old = s
    quality_new, iter_new, struct_new = s_prime
    
    # Recompensa base por mejora de calidad
    quality_improvement = quality_new - quality_old
    
    # Penalización por iteración (tiempo/costo)
    iteration_penalty = -5
    
    # Bonificación por alcanzar producción
    if quality_new >= 85:
        production_bonus = 100
    else:
        production_bonus = 0
    
    # Penalización fuerte por exceder iteraciones
    if iter_new >= 6:
        excess_penalty = -50
    else:
        excess_penalty = 0
    
    # Penalización por empeorar
    if quality_improvement < 0:
        degradation_penalty = quality_improvement * 2  # Doble penalización
    else:
        degradation_penalty = 0
    
    total_reward = (
        quality_improvement + 
        iteration_penalty + 
        production_bonus + 
        excess_penalty + 
        degradation_penalty
    )
    
    return total_reward
```

#### Ejemplos de Recompensas:

**Caso 1: Mejora significativa**
```
s = (45, 1, 'basic')
a = 'add_examples'
s' = (65, 2, 'with_examples')

R = (65-45) + (-5) + 0 + 0 + 0 = +15
```

**Caso 2: Alcanzar producción**
```
s = (75, 3, 'structured')
a = 'add_cot'
s' = (90, 4, 'advanced')

R = (90-75) + (-5) + 100 + 0 + 0 = +110
```

**Caso 3: Empeorar el prompt**
```
s = (55, 2, 'with_examples')
a = 'restructure'
s' = (45, 3, 'structured')

R = (45-55) + (-5) + 0 + 0 + (10*2) = -25
```

**Caso 4: Exceder iteraciones**
```
s = (70, 5, 'structured')
a = 'adjust_temperature'
s' = (72, 6, 'structured')

R = (72-70) + (-5) + 0 + (-50) + 0 = -53
```

### 5. Factor de Descuento (γ)
```
γ = 0.9
```

Un factor alto (0.9) porque queremos que el agente considere recompensas futuras y no solo mejoras inmediatas.

## Política Óptima

Una política heurística razonable sería:
```
π(s):
    if quality_score < 50 and prompt_structure == 'basic':
        return 'add_examples'
    
    elif quality_score < 70 and prompt_structure == 'with_examples':
        return 'restructure'
    
    elif quality_score < 85 and prompt_structure == 'structured':
        return 'add_cot' or 'adjust_temperature'
    
    elif quality_score >= 85:
        return 'deploy'
    
    elif iteration_count >= 5:
        return 'test_edge_cases' (validar antes de decidir)
```

## Ejemplo de Trayectoria Óptima
```
Estado inicial: s0 = (25, 0, 'basic')

s0 = (25, 0, 'basic')
    ↓ [add_examples]
s1 = (45, 1, 'with_examples')     R = +15
    ↓ [restructure]
s2 = (65, 2, 'structured')        R = +15
    ↓ [add_constraints]
s3 = (75, 3, 'structured')        R = +5
    ↓ [add_cot]
s4 = (88, 4, 'advanced')          R = +108
    ↓ [deploy]
s5 = TERMINAL                     R = 0

Recompensa total: 15 + 15 + 5 + 108 = 143
Pasos: 4 iteraciones
```

## Ejemplo de Trayectoria Subóptima
```
Estado inicial: s0 = (25, 0, 'basic')

s0 = (25, 0, 'basic')
    ↓ [adjust_temperature]
s1 = (30, 1, 'basic')             R = 0
    ↓ [simplify] (inválido, asume que se permite)
s2 = (28, 2, 'basic')             R = -7
    ↓ [add_examples]
s3 = (48, 3, 'with_examples')     R = +15
    ↓ [adjust_temperature]
s4 = (52, 4, 'with_examples')     R = -1
    ↓ [restructure]
s5 = (70, 5, 'structured')        R = +13
    ↓ [add_cot]
s6 = (75, 6, 'structured')        R = -48 (exceso)

Recompensa total: 0 - 7 + 15 - 1 + 13 - 48 = -28
Pasos: 6 iteraciones (límite alcanzado sin producción)
```

## Matriz de Transición Simplificada

Para el estado `s = (45, 1, 'with_examples')`:

| Acción | s' (quality, iter, struct) | P(s'\|s,a) | R(s,a,s') |
|--------|---------------------------|------------|-----------|
| add_examples | (50, 2, 'with_examples') | 0.70 | 0 |
| restructure | (65, 2, 'structured') | 0.50 | +15 |
| restructure | (55, 2, 'structured') | 0.30 | +5 |
| restructure | (40, 2, 'structured') | 0.20 | -15 |
| adjust_temperature | (52, 2, 'with_examples') | 0.60 | +2 |
| add_constraints | (58, 2, 'with_examples') | 0.65 | +8 |